In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16
from keras import Sequential
from keras.layers import Dense
import os
from keras.preprocessing import image
import cv2
import random
vgg=VGG16()
model=Sequential()

In [ ]:
categories = ["with_mask","without_mask"]

In [ ]:
data=[]
for category in categories:
    path = os.path.join("train", category)
    
    for file in os.listdir(path):
        img_path = os.path.join(path, file)
        img = cv2.imread(img_path)

        if img is not None:
            img = cv2.resize(img, (224, 224))
            label=categories.index(category)
            print(img.shape)
            data.append([img,label])
        else:
            print(f"Error loading image: {img_path}")

    print(path)


In [ ]:
random.shuffle(data)

In [ ]:
X =[]
y=[]
for features,label in data:
    X.append(features)
    y.append(label)
    

In [ ]:
len(X)

In [ ]:
len(y)


In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
X = X/255

In [ ]:
X[0]

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [ ]:
model.summary()

In [ ]:
for layer in model.layers:
    layer.trainable = False

In [ ]:
model.summary()

In [ ]:
model.add(Dense(1,activation="sigmoid"))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer="Adam",loss="binary_crossentropy",metrics=["accuracy"])

In [ ]:
model.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [ ]:
cap = cv2.VideoCapture(0)

In [ ]:
def detect(img):
    y_pred = model.predict(img.reshape(1,224,224,3))
    return y_pred

In [ ]:
def draw(img,text,pos,bg_color):
    text_size = cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)
    
    end_x = pos[0] + text_size[0][0] + 2
    end_y = pos[1] + text_size[0][1] - 2
    
    cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)

In [ ]:
while True:
    
    ret, frame = cap.read()
    img = cv2.resize(frame,(224,224))
    y_pred = detect(img)
    draw(frame,"Face Mask Detection",(10,10),(255,0,0))
    if y_pred==0:
        draw(frame,"Mask",(30,30),(0,255,0))
    else:
        draw(frame,"No Mask",(30,30),(0,0,255))
        
    cv2.imshow("window",frame)
    
    if cv2.waitKey(1) & 0xFF == ord("x"):
        break
cv2.destroyAllWindows()